# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset **tanpa label** dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.
   
2. **Ketentuan Dataset**:
   - **Tanpa label**: Dataset tidak boleh memiliki label atau kelas.
   - **Jumlah Baris**: Minimal 1000 baris untuk memastikan dataset cukup besar untuk analisis yang bermakna.
   - **Tipe Data**: Harus mengandung data **kategorikal** dan **numerikal**.
     - *Kategorikal*: Misalnya jenis kelamin, kategori produk.
     - *Numerikal*: Misalnya usia, pendapatan, harga.

3. **Pembatasan**:  
   Dataset yang sudah digunakan dalam latihan clustering (seperti customer segmentation) tidak boleh digunakan.

# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

In [2]:
data=pd.read_csv('data_cleaned.csv')
data

FileNotFoundError: [Errno 2] No such file or directory: 'data_cleaned.csv'

# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset. EDA bertujuan untuk:

1. **Memahami Struktur Data**
   - Tinjau jumlah baris dan kolom dalam dataset.  
   - Tinjau jenis data di setiap kolom (numerikal atau kategorikal).

2. **Menangani Data yang Hilang**  
   - Identifikasi dan analisis data yang hilang (*missing values*). Tentukan langkah-langkah yang diperlukan untuk menangani data yang hilang, seperti pengisian atau penghapusan data tersebut.

3. **Analisis Distribusi dan Korelasi**  
   - Analisis distribusi variabel numerik dengan statistik deskriptif dan visualisasi seperti histogram atau boxplot.  
   - Periksa hubungan antara variabel menggunakan matriks korelasi atau scatter plot.

4. **Visualisasi Data**  
   - Buat visualisasi dasar seperti grafik distribusi dan diagram batang untuk variabel kategorikal.  
   - Gunakan heatmap atau pairplot untuk menganalisis korelasi antar variabel.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

**1. Memahami Struktur Data**

In [ ]:
data.describe()

**2. Menangani Data yang Hilang**

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

**3. Analisis Distribusi dan Korelasi**

In [ ]:
data.hist(figsize=(10,8))
plt.show()

In [ ]:
numerical_columns = data.select_dtypes(include=['float64', 'int64']).columns

In [ ]:
for column in numerical_columns:
    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(data[column], kde=True,bins=30,color='blue')
    plt.title(f'Histogram of {column}')
    plt.xlabel(column)
    plt.ylabel('Freqkuensi')

In [ ]:
for column in numerical_columns:
    plt.figure(figsize=(8, 4))
    plt.subplot(1, 2, 2)
    sns.boxplot(y=data[column], color='green')
    plt.title(f"Boxplot: {column}")
    plt.ylabel(column)

    plt.tight_layout()
    plt.show()

In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
sns.histplot(data['unit_price'], kde=True, bins=20, color='skyblue')
plt.title('Distribusi Harga Satuan', fontsize=16)
plt.xlabel('Harga Satuan', fontsize=12)
plt.ylabel('Jumlah', fontsize=12)
plt.show()

plt.figure(figsize=(10, 6))
category_counts = data['category'].value_counts()
sns.barplot(x=category_counts.index.astype(str), y=category_counts.values, palette='viridis')
plt.title('Distribusi Kategori Produk', fontsize=16)
plt.xlabel('Kategori Produk', fontsize=12)
plt.ylabel('Jumlah', fontsize=12)
plt.show()

plt.figure(figsize=(12, 6))
top_products = data.groupby('product')['quantity'].sum().sort_values(ascending=False).head(5)
sns.barplot(x=top_products.index.astype(str), y=top_products.values, palette='coolwarm')
plt.title('5 Produk Terlaris berdasarkan Quantity', fontsize=16)
plt.xlabel('ID Produk', fontsize=12)
plt.ylabel('Jumlah Terjual', fontsize=12)
plt.xticks(rotation=45)
plt.show()

In [ ]:
correlation_matrix = data[numerical_columns].corr()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Heatmap Matriks Korelasi")
plt.show()

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning. Data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang bisa dilakukan, tetapi **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

Cukup sesuaikan dengan karakteristik data yang kamu gunakan yah.

**1. Menghapus atau Menangani Data Kosong (Missing Values)**

In [ ]:
data.drop(columns=['Tanggal'], inplace=True, errors='ignore')

print(data.select_dtypes(include=['object']).columns)

In [ ]:
le = LabelEncoder()
for col in data.select_dtypes(include='object'):
    data[col] = le.fit_transform(data[col])

In [ ]:
missing_values = data.isnull().sum()
print("Missing values sebelum dibersihkan:\n", missing_values)

data_cleaned = data.dropna()

missing_values_after = data_cleaned.isnull().sum()
print("\nMissing values setelah dibersihkan:\n", missing_values_after)

In [ ]:
data_cleaned.info()

In [ ]:
data_cleaned

**2. Menghapus Data Duplikat**

Tidak ada Data yang terduplikat


**3. Normalisasi atau Standarisasi Fitur**

Dikarenakan saya ingin menormalisasikan data kategorikal, jadi saya dahulukan proses encoding

In [ ]:
categorical_columns = ['customer_type', 'product', 'category', 'region', 'cluster']

label_encoders = {col: LabelEncoder() for col in categorical_columns}

for col in categorical_columns:
    data_cleaned[col] = label_encoders[col].fit_transform(data_cleaned[col])

data_cleaned.head()

In [ ]:
numeric_features = ["unit_price", "quantity", "discount", "total_price"]

scaler = StandardScaler()
normalized_features = scaler.fit_transform(data_cleaned[numeric_features])

normalized_df = pd.DataFrame(normalized_features, columns=numeric_features)

normalized_df.head()

**4. Deteksi dan Penanganan Outlier**

In [ ]:
Q1 = data_cleaned['unit_price'].quantile(0.25)
Q3 = data_cleaned['unit_price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_cleaned['unit_price_outlier'] = ((data_cleaned['unit_price'] < lower_bound) |
                                      (data_cleaned['unit_price'] > upper_bound))

In [ ]:
Q1 = data_cleaned['quantity'].quantile(0.25)
Q3 = data_cleaned['quantity'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_cleaned['quantity_outlier'] = ((data_cleaned['quantity'] < lower_bound) |
                                    (data_cleaned['quantity'] > upper_bound))

In [ ]:
Q1 = data_cleaned['discount'].quantile(0.25)
Q3 = data_cleaned['discount'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_cleaned['discount_outlier'] = ((data_cleaned['discount'] < lower_bound) |
                                    (data_cleaned['discount'] > upper_bound))

In [ ]:
Q1 = data_cleaned['total_price'].quantile(0.25)
Q3 = data_cleaned['total_price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

data_cleaned['total_price_outlier'] = ((data_cleaned['total_price'] < lower_bound) |
                                       (data_cleaned['total_price'] > upper_bound))

**5. Bining(Pengelompokan Data)**

In [ ]:
if data_cleaned['region'].dtype == 'object':
    le_region = LabelEncoder()
    data_cleaned['region_encoded'] = le_region.fit_transform(data_cleaned['region'])
else:
    data_cleaned['region_encoded'] = data_cleaned['region']

bin_region_0_10 = data_cleaned["region_encoded"][(data_cleaned["region_encoded"] >= 0) & (data_cleaned["region_encoded"] <= 10)]
bin_region_11_20 = data_cleaned["region_encoded"][(data_cleaned["region_encoded"] >= 11) & (data_cleaned["region_encoded"] <= 20)]
bin_region_21_30 = data_cleaned["region_encoded"][(data_cleaned["region_encoded"] >= 21) & (data_cleaned["region_encoded"] <= 30)]
bin_region_31_40 = data_cleaned["region_encoded"][(data_cleaned["region_encoded"] >= 31) & (data_cleaned["region_encoded"] <= 40)]

region_bins = ["0-10", "11-20", "21-30", "31-40"]
region_counts = [len(bin_region_0_10), len(bin_region_11_20), len(bin_region_21_30), len(bin_region_31_40)]

plt.figure(figsize=(15, 6))
plt.bar(region_bins, region_counts, color=['red', 'green', 'blue', 'cyan'])
plt.title("Distribusi Region Berdasarkan Rentang Kode")
plt.xlabel("Region (Encoded)")
plt.ylabel("Jumlah Peserta")
plt.xticks(rotation=45)
for i in range(len(region_bins)):
    plt.text(i, region_counts[i], region_counts[i], ha='center', va='bottom')

plt.show()

In [ ]:
bin_qty_0_10 = data_cleaned["quantity"][(data_cleaned["quantity"] >= 0) & (data_cleaned["quantity"] <= 10)]
bin_qty_11_20 = data_cleaned["quantity"][(data_cleaned["quantity"] >= 11) & (data_cleaned["quantity"] <= 20)]
bin_qty_21_30 = data_cleaned["quantity"][(data_cleaned["quantity"] >= 21) & (data_cleaned["quantity"] <= 30)]

kategori = ["0-10", "11-20", "21-30"]
jumlah = [len(bin_qty_0_10), len(bin_qty_11_20), len(bin_qty_21_30)]

plt.figure(figsize=(15, 6))
plt.bar(kategori, jumlah, color=['red', 'green', 'blue'])
plt.title("Distribusi Jumlah Produk (Quantity)")
plt.xlabel("Rentang Quantity")
plt.ylabel("Jumlah Peserta")
plt.xticks(rotation=45)

for i in range(len(kategori)):
    plt.text(i, jumlah[i], jumlah[i], ha='center', va='bottom')

plt.show()

# **6. Pembangunan Model Clustering**

## **a. Pembangunan Model Clustering**

Pada tahap ini, Anda membangun model clustering dengan memilih algoritma yang sesuai untuk mengelompokkan data berdasarkan kesamaan. Berikut adalah **rekomendasi** tahapannya.
1. Pilih algoritma clustering yang sesuai.
2. Latih model dengan data menggunakan algoritma tersebut.

In [ ]:
X = data_cleaned[['unit_price', 'total_price']]

kmeans = KMeans(n_clusters=3, random_state=42)

kmeans.fit(X)

## **b. Evaluasi Model Clustering**

Untuk menentukan jumlah cluster yang optimal dalam model clustering, Anda dapat menggunakan metode Elbow atau Silhouette Score.

Metode ini membantu kita menemukan jumlah cluster yang memberikan pemisahan terbaik antar kelompok data, sehingga model yang dibangun dapat lebih efektif. Berikut adalah **rekomendasi** tahapannya.
1. Gunakan Silhouette Score dan Elbow Method untuk menentukan jumlah cluster optimal.
2. Hitung Silhouette Score sebagai ukuran kualitas cluster.

**1. Menggunakan Elbow Method**

In [ ]:
kmeans = KMeans()

visualizer = KElbowVisualizer(kmeans, k=(1, 10))

visualizer.fit(X)

visualizer.show()

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)

data_cleaned['Cluster'] = kmeans.labels_

centroid = kmeans.cluster_centers_

plt.figure(figsize=(10, 6))
sns.scatterplot(data=data_cleaned, x="unit_price", y="total_price", hue="Cluster", palette="Set1", s=50)
plt.scatter(centroid[:, 0], centroid[:, 1], c="black", marker="X", s=200, label="Centroids")
plt.title("Clustering Result (unit_price vs total_price)", fontsize=14)
plt.xlabel("Unit Price")
plt.ylabel("Total Price")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3)
plt.grid()
plt.show()

**2. Menghitung Silhoutte score**

In [ ]:
silhouette_avg = silhouette_score(X,kmeans.labels_)
silhouette_avg

## **c. Feature Selection (Opsional)**

Silakan lakukan feature selection jika Anda membutuhkan optimasi model clustering. Jika Anda menerapkan proses ini, silakan lakukan pemodelan dan evaluasi kembali menggunakan kolom-kolom hasil feature selection. Terakhir, bandingkan hasil performa model sebelum dan sesudah menerapkan feature selection.

In [ ]:
#Type your code here

## **d. Visualisasi Hasil Clustering**

Setelah model clustering dilatih dan jumlah cluster optimal ditentukan, langkah selanjutnya adalah menampilkan hasil clustering melalui visualisasi.

Berikut adalah **rekomendasi** tahapannya.
1. Tampilkan hasil clustering dalam bentuk visualisasi, seperti grafik scatter plot atau 2D PCA projection.

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

kmeans_pca = KMeans(n_clusters=2, random_state=42)
kmeans_pca.fit(X_pca)

plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=kmeans_pca.labels_, palette="Set1", s=50)
plt.scatter(kmeans_pca.cluster_centers_[:, 0], kmeans_pca.cluster_centers_[:, 1],
            c="black", marker="X", s=200, label="Centroids")
plt.title("Hasil Cluster(PCA Projection)", fontsize=14)
plt.xlabel("Komponen 1")
plt.ylabel("Komponen 2")
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3)
plt.grid()
plt.show()

## **e. Analisis dan Interpretasi Hasil Cluster**

Setelah melakukan clustering, langkah selanjutnya adalah menganalisis karakteristik dari masing-masing cluster berdasarkan fitur yang tersedia.

Berikut adalah **rekomendasi** tahapannya.
1. Analisis karakteristik tiap cluster berdasarkan fitur yang tersedia (misalnya, distribusi nilai dalam cluster).
2. Berikan interpretasi: Apakah hasil clustering sesuai dengan ekspektasi dan logika bisnis? Apakah ada pola tertentu yang bisa dimanfaatkan?

**1. Analisis Karakteristik Tiap Cluster**

In [ ]:
data_cleaned['Cluster'] = kmeans_pca.labels_

cluster_summary = data_cleaned.groupby('Cluster')[['unit_price', 'total_price', 'quantity', 'discount']].mean()
print(cluster_summary)

**Hasil Ringkasan:**

Cluster,	unit_price,	total_price,	quantity,	discount

0,	Lebih tinggi,	Lebih tinggi,	Lebih besar,	Lebih rendah

1,	Lebih rendah,	Lebih rendah,	Lebih kecil,	Lebih tinggi

Cluster 0: Transaksi besar, pelanggan membeli dalam jumlah banyak, harga tinggi, dan diskon rendah → kemungkinan pelanggan loyal atau premium.

Cluster 1: Transaksi kecil, jumlah pembelian sedikit, harga rendah, namun mendapatkan diskon lebih tinggi → bisa jadi pelanggan baru atau reguler yang sensitif terhadap harga.

**2. Interpretasi: Apakah hasil clustering sesuai dengan ekspektasi dan logika bisnis?**

Ya, hasil clustering ini sangat relevan secara bisnis dan sesuai dengan ekspektasi. Clustering berhasil memisahkan dua tipe perilaku pelanggan:

- Pelanggan dengan volume pembelian tinggi tapi diskon kecil → cocok ditargetkan untuk loyalty program atau upselling.

- Pelanggan dengan diskon tinggi dan pembelian kecil → bisa diarahkan ke strategi promosi atau edukasi produk untuk mendorong pembelian lebih banyak.

Hal ini menunjukkan bahwa clustering bisa dimanfaatkan untuk:

- Segmentasi pelanggan

- Pengambilan keputusan marketing

- Personalized offer

Tulis hasil interpretasinya di sini.
1. Cluster 1:
   - Berisi pelanggan dengan unit_price dan total_price tinggi.
   - Membeli dalam jumlah besar (quantity besar) dan menerima sedikit diskon.
   - Kemungkinan adalah pelanggan loyal atau premium.

2. Cluster 2:
   - Berisi pelanggan dengan total transaksi kecil dan lebih banyak diskon.
   - Quantity lebih rendah.
   - Kemungkinan pelanggan baru atau pembeli musiman.

# **7. Mengeksport Data**

Simpan hasilnya ke dalam file CSV.

In [ ]:
data_cleaned.to_csv("data_clustering.csv", index=False)